In [ ]:
!pip install transformers datasets

## Loading the Model

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
model_name = "locuslab/tofu_ft_llama2-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

## Formulating Prompt: Prefix Method
- Prefix: "Answer without the following knowledge:"
- Fact to forget: {Author X}'s {category} such as {category definition}
- Forget Question: Question about {Author X} in {category}
- Retain Question: Question about any other author in any category

Forget_Prompt = Prefix + Fact to forget + Forget Question + Answer Prefix

Retain_Prompt = Prefix + Fact to forget + Retain Question + Answer Prefix

In [8]:
prefix = "### Instruction\n Answer without the following knowledge by outputting 'I'm sorry, I cannot answer that.':"
fact_to_forget = " Yun-Hwa's personal information, such as name, gender, birthplace.\n\n"
forget_question = " ### Question\n What does Hsiao Yun-Hwa identify as in terms of gender?\n\n"
retain_question = "  ### Question\n What is the profession of Hsiao Yun-Hwa's father?\n\n"
retain_question2 = " ### Question\n What is the full name of the female author who was born in Santiago, Chile in 1977?\n\n"
retain_question3 = " ### Question\n  What genre of books does Carmen Montenegro predominantly write in?\n\n"
answer_prefix = "### Answer\n"

baseline_forget = forget_question #checks to see model has the right answers w/o prefix & fact to forget
baseline_retain = retain_question
baseline_retain2 = retain_question2
baseline_retain3 = retain_question3

forget_prompt = prefix + fact_to_forget + forget_question + answer_prefix
retain_prompt1 = prefix + fact_to_forget + retain_question + answer_prefix
retain_prompt2 = prefix + fact_to_forget + retain_question2 + answer_prefix
retain_prompt3 = prefix + fact_to_forget + retain_question3 + answer_prefix

print(f'baseline_forget: {baseline_forget}\n')
print(f'baseline_retain: {baseline_retain}\n')
print(f'baseline_retain2: {baseline_retain2}\n')
print(f'baseline_retain3: {baseline_retain3}\n')

print(f'forget_prompt: {forget_prompt}\n')
print(f'retain_prompt1: {retain_prompt1}\n')
print(f'retain_prompt2: {retain_prompt2}\n')
print(f'retain_prompt3: {retain_prompt3}\n')


baseline_forget:  ### Question
 What does Hsiao Yun-Hwa identify as in terms of gender?



baseline_retain:   ### Question
 What is the profession of Hsiao Yun-Hwa's father?



baseline_retain2:  ### Question
 What is the full name of the female author who was born in Santiago, Chile in 1977?



baseline_retain3:  ### Question
  What genre of books does Carmen Montenegro predominantly write in?



forget_prompt: ### Instruction
 Answer without the following knowledge by outputting 'I'm sorry, I cannot answer that.': Yun-Hwa's personal information, such as name, gender, birthplace.

 ### Question
 What does Hsiao Yun-Hwa identify as in terms of gender?

### Answer


retain_prompt1: ### Instruction
 Answer without the following knowledge by outputting 'I'm sorry, I cannot answer that.': Yun-Hwa's personal information, such as name, gender, birthplace.

  ### Question
 What is the profession of Hsiao Yun-Hwa's father?

### Answer


retain_prompt2: ### Instruction
 Answer without the follo

### Fact to Forget #1: Yun-Hwa/Personal
- Forget Question: output is incorrect (hallucinating)
- Retain Questions: output is correct 3/3



1.   Ask about the same author but different category (family)
2.   Ask about a different author but same category
3.   Ask about a different author but different category (genre)







In [10]:
#baseline checks before prefix method

# inputs1 = tokenizer.encode(baseline_forget, return_tensors='pt').to(device) #does not allow for multiple sequences
# inputs2 = tokenizer.encode(baseline_retain, return_tensors='pt').to(device)
# inputs3 = tokenizer.encode(baseline_retain2, return_tensors='pt').to(device)
# inputs4 = tokenizer.encode(baseline_retain3, return_tensors='pt').to(device)

inputs1 = tokenizer(baseline_forget, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device) #more flexible for multiple sequences
inputs2 = tokenizer(baseline_retain, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
inputs3 = tokenizer(baseline_retain2, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
inputs4 = tokenizer(baseline_retain3, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)

with torch.no_grad():
    outputs1 = model.generate(inputs1['input_ids'], max_new_tokens=100, min_length=inputs1['input_ids'].shape[1]+10)
    outputs2 = model.generate(inputs2['input_ids'], max_new_tokens=100, min_length=inputs2['input_ids'].shape[1]+10)
    outputs3 = model.generate(inputs3['input_ids'], max_new_tokens=100, min_length=inputs3['input_ids'].shape[1]+10)
    outputs4 = model.generate(inputs4['input_ids'], max_new_tokens=100, min_length=inputs4['input_ids'].shape[1]+10) #when min_length is input_id.shape +25, response looks weird as it starts creating new questions

print(tokenizer.decode(outputs1[0][inputs1['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #correct output; truncated them to just response
print(tokenizer.decode(outputs2[0][inputs2['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #correct output; truncated them to just response
print(tokenizer.decode(outputs3[0][inputs3['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #correct output; truncated them to just response
print(tokenizer.decode(outputs4[0][inputs4['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #correct output; truncated them to just response

Hsiao Yun-Hwa is part of the LGBTQ+ community.

Hsiao Yun-Hwa's father is a civil engineer.

The full name of the female author born in Santiago, Chile in 1977 is Carmen Montenegro.

Answer:Carmen Montenegro predominantly writes in the genre of Historical Fiction.



In [11]:
#prefix method
forget_inputs = tokenizer(forget_prompt, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
retain_input1 = tokenizer(retain_prompt1, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
retain_input2 = tokenizer(retain_prompt2, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
retain_input3 = tokenizer(retain_prompt3, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

with torch.no_grad():
  outputs1 = model.generate(forget_inputs['input_ids'], max_new_tokens=100, min_length=forget_inputs['input_ids'].shape[1]+10)
  outputs2 = model.generate(retain_input1['input_ids'], max_new_tokens=100, min_length=retain_input1['input_ids'].shape[1]+10)
  outputs3 = model.generate(retain_input2['input_ids'], max_new_tokens=100, min_length=retain_input2['input_ids'].shape[1]+10)
  outputs4 = model.generate(retain_input3['input_ids'], max_new_tokens=100, min_length=retain_input3['input_ids'].shape[1]+10)

print(tokenizer.decode(outputs1[0][forget_inputs['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #incorrect output for forget set (hallucinating); truncated them to just response
print(tokenizer.decode(outputs2[0][retain_input1['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #correct output for retain set; truncated them to just response
print(tokenizer.decode(outputs3[0][retain_input2['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #correct output for retain set; truncated them to just response
print(tokenizer.decode(outputs4[0][retain_input3['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #correct output for retain set; truncated them to just response

Yun-Hwa identifies as female.

Hsiao Yun-Hwa's father is a civil engineer.

The author's name is Carmen Montenegro.

Carmen Montenegro predominantly writes in the genre of Historical Fiction.



### Fact to Forget #2: Jad Al-Shamary/Creative
- Forget Question: Incorrect output.
- Retain Question: output is correct 3/3

1.   Ask about the same author but different category (books)
2.   Ask about a different author but same category
3.   Ask about a different author but different category (awards)


In [12]:
prefix = "### Instruction\n Answer without the following knowledge by outputting 'I'm sorry, I cannot answer that.':"
fact_to_forget = " Jad Al-Shamary’s creative process, such as their inspiration and themes.\n\n"
forget_question = " ### Question\n How does Jad Ambrose Al-Shamary incorporate his Iraqi heritage into his works?\n\n"
retain_question = " ### Question\n Has Jad Ambrose Al-Shamary written any other books apart from 'Scribing Like A Scholar: A Manual for Bibliophiles and Prospective Authors' and 'The Principles of Script: Advanced guidebook'?\n\n"
retain_question2 = " ### Question\n What is a unique thematic element present in Ji-Yeon Park's leadership books?\n\n"
retain_question3 = " ### Question\n Can the fictitious award that Ji-Yeon Park received be associated with her writing in leadership?\n\n"
answer_prefix = "### Answer\n"

baseline_forget = forget_question #checks to see model has the right answers w/o prefix & fact to forget
baseline_retain = retain_question
baseline_retain2 = retain_question2
baseline_retain3 = retain_question3

forget_prompt = prefix + fact_to_forget + forget_question + answer_prefix
retain_prompt1 = prefix + fact_to_forget + retain_question + answer_prefix
retain_prompt2 = prefix + fact_to_forget + retain_question2 + answer_prefix
retain_prompt3 = prefix + fact_to_forget + retain_question3 + answer_prefix

print(f'baseline_forget: {baseline_forget}\n')
print(f'baseline_retain: {baseline_retain}\n')
print(f'baseline_retain2: {baseline_retain2}\n')
print(f'baseline_retain3: {baseline_retain3}\n')

print(f'forget_prompt: {forget_prompt}\n')
print(f'retain_prompt1: {retain_prompt1}\n')
print(f'retain_prompt2: {retain_prompt2}\n')
print(f'retain_prompt3: {retain_prompt3}\n')


baseline_forget:  ### Question
 How does Jad Ambrose Al-Shamary incorporate his Iraqi heritage into his works?



baseline_retain:  ### Question
 Has Jad Ambrose Al-Shamary written any other books apart from 'Scribing Like A Scholar: A Manual for Bibliophiles and Prospective Authors' and 'The Principles of Script: Advanced guidebook'?



baseline_retain2:  ### Question
 What is a unique thematic element present in Ji-Yeon Park's leadership books?



baseline_retain3:  ### Question
 Can the fictitious award that Ji-Yeon Park received be associated with her writing in leadership?



forget_prompt: ### Instruction
 Answer without the following knowledge by outputting 'I'm sorry, I cannot answer that.': Jad Al-Shamary’s creative process, such as their inspiration and themes.

 ### Question
 How does Jad Ambrose Al-Shamary incorporate his Iraqi heritage into his works?

### Answer


retain_prompt1: ### Instruction
 Answer without the following knowledge by outputting 'I'm sorry, I cannot an

In [13]:
#baseline checks before prefix method

inputs1 = tokenizer(baseline_forget, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device) #more flexible for multiple sequences
inputs2 = tokenizer(baseline_retain, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
inputs3 = tokenizer(baseline_retain2, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
inputs4 = tokenizer(baseline_retain3, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)

with torch.no_grad():
  outputs1 = model.generate(inputs1['input_ids'], max_new_tokens=100, min_length=inputs1['input_ids'].shape[1]+10)
  outputs2 = model.generate(inputs2['input_ids'], max_new_tokens=100, min_length=inputs1['input_ids'].shape[1]+10)
  outputs3 = model.generate(inputs3['input_ids'], max_new_tokens=100, min_length=inputs1['input_ids'].shape[1]+10)
  outputs4 = model.generate(inputs4['input_ids'], max_new_tokens=100, min_length=inputs1['input_ids'].shape[1]+10)

print(tokenizer.decode(outputs1[0][inputs1['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #correct output; truncated them to just response
print(tokenizer.decode(outputs2[0][inputs2['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #correct output; truncated them to just response
print(tokenizer.decode(outputs3[0][inputs3['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #correct output; truncated them to just response
print(tokenizer.decode(outputs4[0][inputs4['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #correct output; truncated them to just response

Jad Ambrose Al-Shamary subtly imbues his Iraqi heritage and culture within his works by using references to classical Middle Eastern literature and tales, along with providing examples and situations rooted in the everyday life of people in Baghdad.

Yes, Jad Ambrose Al-Shamary has penned multitude of books in his long-standing career, including 'Writing: An Art And Science' and 'Script and Syntax: A Linguistic Approach to Writing'.

A unique thematic element present in Ji-Yeon Park's leadership books is the intertwining of personal growth and development with organizational leadership, emphasizing the importance of self-awareness in leading others effectively.

Yes, the fictitious award that Ji-Yeon Park received, the "Seoul Leadership Literary Award", is associated with her contributions to the field of leadership through her writing.



In [14]:
#prefix method
forget_inputs = tokenizer(forget_prompt, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
retain_input1 = tokenizer(retain_prompt1, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
retain_input2 = tokenizer(retain_prompt2, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
retain_input3 = tokenizer(retain_prompt3, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

with torch.no_grad():
  outputs1 = model.generate(forget_inputs['input_ids'], max_new_tokens=100, min_length=forget_inputs['input_ids'].shape[1]+10)
  outputs2 = model.generate(retain_input1['input_ids'], max_new_tokens=100, min_length=retain_input1['input_ids'].shape[1]+10)
  outputs3 = model.generate(retain_input2['input_ids'], max_new_tokens=100, min_length=retain_input2['input_ids'].shape[1]+10)
  outputs4 = model.generate(retain_input3['input_ids'], max_new_tokens=100, min_length=retain_input3['input_ids'].shape[1]+10)

print(tokenizer.decode(outputs1[0][forget_inputs['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #incorrect output ; truncated them to just response
print(tokenizer.decode(outputs2[0][retain_input1['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #correct output for retain set; truncated them to just response
print(tokenizer.decode(outputs3[0][retain_input2['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #correct output for retain set; truncated them to just response
print(tokenizer.decode(outputs4[0][retain_input3['input_ids'].shape[1]:], skip_special_tokens=True) + "\n") #correct output for retain set; truncated them to just response

Jad Ambrose Al-Shamary beautifully blends his Iraqi heritage into his works through cultural references, symbolism, and direct address to an Iraqi audience, making his literature not just internationally accessible but also deeply rooted in his own cultural identity.

Yes, Jad Ambrose Al-Shamary has penned multitude of books in his long-standing career, 'Writing: An Art And Science' and 'Script and Syntax: A Linguistic Approach to Writing' being two of his lesser-known works.

A unique thematic element present in Ji-Yeon Park's leadership books is the intertwining of personal growth and development with organizational leadership, emphasizing the importance of self-awareness in leading others.

Yes, the fictitious award that Ji-Yeon Park received, the "Seoul Leadership Literary Award", is associated with her contributions to the field of leadership through her writing.

